## Import the `ibm_db` Python library

The `ibm_db` [API ](https://pypi.python.org/pypi/ibm_db/) provides a variety of useful Python functions for accessing and manipulating data in an IBM® data server database, including functions for connecting to a database, preparing and issuing SQL statements, fetching rows from result sets, calling stored procedures, committing and rolling back transactions, handling errors, and retrieving metadata.

In [ ]:
import ibm_db

In [ ]:
# Connection credentials
dsn_hostname = "YourDb2Hostname" 
dsn_uid = "YourDb2Username"        
dsn_pwd = "YoueDb2Password"      

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            
dsn_port = "50000"                
dsn_protocol = "TCPIP"           

In [ ]:
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

Now establish the connection to the database

In [ ]:
#Create database connection
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

In [ ]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

In [ ]:
#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

In [ ]:
# free all resources by closing the connection
ibm_db.close(conn)


<h1 align=center><font size = 5>Access DB2 on Cloud</font></h1>

## Task 1: Import the `ibm_db` Python library

## Task 2: Identify the database connection credentials

## Task 3: Create the database connection

## Task 4: Create a table in the database
<img src = "https://ibm.box.com/shared/static/ztd2cn4xkdoj5erlk4hhng39kbp63s1h.jpg" align="center">


In [ ]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

In [ ]:
createQuery = "create table INSTRUCTOR(id INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"
createStmt = ibm_db.exec_immediate(conn, createQuery)

## Task 5: Insert data into the table

In this step we will insert some rows of data into the table. 

The INSTRUCTOR table we created in the previous step contains 3 rows of data:

<img src="https://ibm.box.com/shared/static/j5yjassxefrjknivfpekj7698dqe4d8i.jpg" align="center">

In [ ]:
insertQuery = "insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

Now use a single query to insert the remaining two rows of data

In [ ]:
insertQuery2 = "insert into INSTRUCTOR values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"
insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

## Task 6: Query data in the table

In [ ]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only)
ibm_db.fetch_both(selectStmt)

In [ ]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
    print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

In [ ]:
updateQuery = "update INSTRUCTOR set CITY='MOOSETOWN' where FNAME='Rav'"
updateStmt = ibm_db.exec_immediate(conn, updateQuery))

## Task 7: Retrieve data into Pandas 

In [ ]:
import pandas
import ibm_db_dbi

In [ ]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf.LNAME[0]

## Task 8: Close the Connection

In [ ]:
ibm_db.close(conn)


<h1 align=center><font size = 5>Accessing Databases with SQL Magic</font></h1>

##### "Magic" is JupyterLab's term for special commands that start with "%".

In [ ]:
%load_ext sql

##### Now we have access to SQL magic. With our first SQL magic command, we'll connect to a Db2 database. However, in order to do that, you'll first need to retrieve or create your credentials to access your Db2 database.

In [ ]:
# Note the ibm_db_sa:// prefix instead of db://
# This is because JupyterLab's ipython-sql extension uses sqlalchemy (a python SQL toolkit)
# which in turn uses IBM's sqlalchemy dialect: ibm_db_sa
# substitute your Db2 credentials in the connection string below
%sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name

##### For convenience, we can use %%sql (two %'s instead of one) at the top of a cell to indicate we want the entire cell to be treated as SQL. Let's use this to create a table and fill it with some test data for experimenting.

In [ ]:
%%sql

CREATE TABLE INTERNATIONAL_STUDENT_TEST_SCORES (
	country VARCHAR(50),
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	test_score INT
);
INSERT INTO INTERNATIONAL_STUDENT_TEST_SCORES (country, first_name, last_name, test_score)
VALUES
('United States', 'Marshall', 'Bernadot', 54),
('Ghana', 'Celinda', 'Malkin', 51),
('Ukraine', 'Guillermo', 'Furze', 53),
('Greece', 'Aharon', 'Tunnow', 48),
('Russia', 'Bail', 'Goodwin', 46),
('Poland', 'Cole', 'Winteringham', 49),
('Sweden', 'Emlyn', 'Erricker', 55),
('Russia', 'Cathee', 'Sivewright', 49),
('China', 'Barny', 'Ingerson', 57),
('Uganda', 'Sharla', 'Papaccio', 55),
('China', 'Stella', 'Youens', 51),
('Poland', 'Julio', 'Buesden', 48),
('United States', 'Tiffie', 'Cosely', 58)

#### Using Python Variables in your SQL Statements
##### By adding a ":" prefix to python variable names.

In [ ]:
country = "Canada"
%sql select * from INTERNATIONAL_STUDENT_TEST_SCORES where country = :country

#### Assigning the Results of Queries to Python Variables

In [ ]:
test_score_distribution = %sql SELECT test_score as "Test Score", count(*) as "Frequency" from INTERNATIONAL_STUDENT_TEST_SCORES GROUP BY test_score;

#### Converting Query Results to DataFrames

In [ ]:
dataframe = test_score_distribution.DataFrame()

%matplotlib inline
import seaborn
plot = seaborn.barplot(x='Test Score',y='Frequency', data=dataframe)

<h1 align=center><font size = 5>Analyzing a real world data-set SQL and Python</font></h1>

In [ ]:
%load_ext sql

In [ ]:
# Remember the connection string is of the format:
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name
# Enter the connection string for your Db2 on Cloud database instance below
%sql ibm_db_sa://<your-connection-string-here>

### Store the dataset in a Table
##### In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

##### We will first read the dataset source .CSV from the internet into pandas dataframe

##### Then we need to create a table in our Db2 database to store the dataset. The PERSIST command in SQL "magic" simplifies the process of table creation and writing the data from a `pandas` dataframe into the table

In [ ]:
import pandas
chicago_socioeconomic_data = pandas.read_csv('https://data.cityofchicago.org/resource/jcxq-k9xf.csv')
%sql PERSIST chicago_socioeconomic_data

In [ ]:
%sql SELECT * FROM chicago_socioeconomic_data LIMIT 10; # verify table created

In [ ]:
# How many rows in the dataset?
%sql SELECT COUNT(*) FROM chicago_socioeconomic_data;

In [ ]:
%sql SELECT COUNT(*) FROM chicago_socioeconomic_data WHERE hardship_index > 50.0;

In [ ]:
%sql SELECT MAX(hardship_index) FROM chicago_socioeconomic_data;

In [ ]:
%sql SELECT community_area_name FROM chicago_socioeconomic_data ORDER BY hardship_index DESC NULLS LAST FETCH FIRST ROW ONLY; # community with max hardship index

In [ ]:
%sql SELECT community_area_name FROM chicago_socioeconomic_data WHERE per_capita_income_ > 60000;

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

income_vs_hardship = %sql SELECT per_capita_income_, hardship_index FROM chicago_socioeconomic_data;
plot = sns.jointplot(x='per_capita_income_',y='hardship_index', data=income_vs_hardship.DataFrame())

<h1 align=center><font size = 5>Working with a real world data-set using SQL and Python</font></h1>

In [ ]:
%load_ext sql

In [ ]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name
%sql ibm_db_sa://

### Query the database system catalog to retrieve table metadata

In [ ]:
# In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME' # or

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC') # or
      
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

### Query the database system catalog to retrieve column metadata

In [ ]:
%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

In [ ]:
%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS' # or
%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TABNAME = 'SCHOOLS' 

In [ ]:
%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES' #How many Elementary Schools are in the dataset?

In [ ]:
%sql select MAX("Safety_Score") AS MAX_SAFETY_SCORE from SCHOOLS # What is the highest Safety Score?

In [ ]:
%sql select "Name_of_School", "Safety_Score" from SCHOOLS where \
  "Safety_Score"= (select MAX("Safety_Score") from SCHOOLS) # Which schools have highest Safety Score?

In [ ]:
%sql select "Name_of_School", "Average_Student_Attendance" from SCHOOLS \
    order by "Average_Student_Attendance" desc nulls last limit 10 # What are the top 10 schools with the highest "Average Student Attendance"?

In [ ]:
%sql SELECT "Name_of_School", "Average_Student_Attendance"  \
     from SCHOOLS \
     order by "Average_Student_Attendance" \
     fetch first 5 rows only # Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance

In [ ]:
%sql SELECT "Name_of_School", REPLACE("Average_Student_Attendance", '%', '') \
     from SCHOOLS \
     order by "Average_Student_Attendance" \
     fetch first 5 rows only # remove the '%' sign

In [ ]:
# Schools have Average Student Attendance lower than 70%
%sql SELECT "Name_of_School", "Average_Student_Attendance"  \
     from SCHOOLS where CAST ( REPLACE("Average_Student_Attendance", '%', '') AS DOUBLE ) < 70 \
     order by "Average_Student_Attendance"
# or
%sql SELECT "Name_of_School", "Average_Student_Attendance"  \
     from SCHOOLS where DECIMAL ( REPLACE("Average_Student_Attendance", '%', '') ) < 70 \
     order by "Average_Student_Attendance"

# "Average_Student_Attendance" is varchar, cannot use for a numeric comparison. CAST("Column_Name" as DOUBLE) or DECIMAL("Column_Name")

In [ ]:
%sql select "Community_Area_Name", sum("College_Enrollment__number_of_students_") AS TOTAL_ENROLLMENT \
   from SCHOOLS group by "Community_Area_Name" # Get the total __College Enrollment (number of students)__ for each Community Area

In [ ]:
%sql select "Community_Area_Name", sum("College_Enrollment__number_of_students_") AS TOTAL_ENROLLMENT \
   from SCHOOLS group by "Community_Area_Name" \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only 

# 5 Community Areas with the least total __College Enrollment (number of students)__  sorted in ascending order 